In [1]:
from __future__ import print_function
import argparse
import geopandas as gpd
import json
import numpy as np
import math
from math import pi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import pandas as pd
import pprint
import pylab as pl
import requests
from scipy.cluster.hierarchy import linkage, dendrogram, set_link_color_palette
import seaborn as sns
from sklearn import preprocessing
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
import shutil 
import sys
import urllib
import urllib.request 
import zipfile
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode
    
from pandas.io.json import json_normalize
from shapely.geometry import Point
from ast import literal_eval

In [2]:
# Dummy test to see if api responds correctly 

#API_KEY = open('yelp_api.txt', 'r').readlines()[1][:-1] # api stored in seperate .txt file
API_KEY='ZBNLt73SjM2I2rKuqJTTnuDIL3WQ3mD3hBeQiQZXkmEsoW-eOMZXEVQ8IOXcj0M35H3KDBU3GVnrl22OlE1ONbLpxgQW7aTa_cffBHZt3qqbjPrHAHk_wf0EZ8DIXHYx'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
#url=API_HOST+SEARCH_PATH
#print(url)
#BUSINESS_PATH = '/v3/businesses/'
price=[1,2,3]
### Search Terms
params = {
    'term': 'coffee',
    'limit':'50',
    'price':price,
    'location': 'MI'
}
url_params=params
host=API_HOST
path=SEARCH_PATH
api_key=API_KEY
url = '{0}{1}'.format(host, quote(path.encode('utf8')))
print(url)
headers = {
    'Authorization': 'Bearer %s' % api_key,
}
response = requests.request('GET', url, headers=headers, params=url_params)
data=response.json()


https://api.yelp.com/v3/businesses/search


In [3]:
print(data['businesses'])

[{'id': '70Px6qdqnqD_cH_0_WUZnQ', 'alias': 'astro-coffee-detroit', 'name': 'Astro Coffee', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/alQ7prY_yRD8nz1CT_-Lpw/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/astro-coffee-detroit?adjust_creative=x9NjtX_2kQXZJ0xLpaTY-A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=x9NjtX_2kQXZJ0xLpaTY-A', 'review_count': 268, 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}], 'rating': 4.5, 'coordinates': {'latitude': 42.3315706327176, 'longitude': -83.0757039067459}, 'transactions': [], 'price': '$$', 'location': {'address1': '2124 Michigan Ave', 'address2': '', 'address3': '', 'city': 'Detroit', 'zip_code': '48216', 'country': 'US', 'state': 'MI', 'display_address': ['2124 Michigan Ave', 'Detroit, MI 48216']}, 'phone': '+13136382989', 'display_phone': '(313) 638-2989', 'distance': 3652.4494388608036}, {'id': '5hGsCVDQ_Iz6997CXG3amw', 'alias': 'roasting-plant-detroit-detroit', 'name': 'Roasting Pla

In [4]:
#useful definitions
def request(host=API_HOST, path=SEARCH_PATH, api_key=API_KEY, url_params=params):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [5]:
def search(api_key, term, location, categories, offset, price):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': int(params['limit']),
        'offset': offset,
        'categories': categories,
        'price':price
    }
    
    find_locs = request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)
    try:
        return json_normalize(find_locs['businesses'])
    except AttributeError:
        return find_locs

In [6]:
#clean up initial pull from the dictionary and create an easily understandable dataframe
def filtered_coffee(df):
    '''
    Returns dataframe with only necessary columns, and cleaned in such a way as we will need them later.
    '''
    category_list = [[df['categories'][j][i]['title'] \
    for i in range(len(df['categories'][j]))] \
    for j in range(len(df['categories']))]

    location = list(zip(df['coordinates.latitude'], 
                             df['coordinates.longitude']))

    columns = ['Store_Name', 'Categories','B_Id' ,'Location', 'City', 
                   'Zip_Code', 'Price', 'Rating', 'Review_Count']
    data = [list(df['name']), category_list,df['id'], location, list(df['location.city']),
                list(df['location.zip_code']), list(df['price']), list(df['rating']), 
                list(df['review_count'])]


    filtered_coffee = pd.DataFrame(data = dict(zip(columns, data)))
    filtered_coffee = filtered_coffee[columns]
    return filtered_coffee


In [7]:
## USED TO GET NEIGHBORHOOD NAMES FOR NEW CITIES
def find_coffee(search_term='coffee', loc='Brooklyn',category=None, loops=20, price=[1,2,3,4]):
    
    '''
    Pulls all of the coffee shops from yelp, doing many exhausting cycles since .
    '''
    
    coffee_shops_df = None
    print(loc)
        
    for i in range(loops):

        try:
            # max api allows is 50 per pull
            coffee_results = search(API_KEY, search_term, loc, category, i*50, price)
        except IndexError:
            # error raised once no more coffee shops in the area are found.
            # this loop allows to capture the largest remaining no. of coffee shops
            for j in range(49, -1, -1): 
                if j == 0:
                    break
                try:
                    coffee_results = search(API_KEY, search_term, loc, category, i*j, price)
                except IndexError:
                    continue
                break

        try:
            coffee_results = filtered_coffee(coffee_results)

            if isinstance(coffee_shops_df, pd.DataFrame):
                coffee_shops_df = pd.concat([coffee_shops_df, coffee_results])
                coffee_shops_df.reset_index(drop=True, inplace=True)
            else:
                coffee_shops_df = coffee_results
        except KeyError:
            break
        
#     coffee_shops_df = coffee_shops_df.drop_duplicates('Location')
#     coffee_shops_df = coffee_shops_df[coffee_shops_df['Review_Count'] > 5]
#     coffee_shops_df = coffee_shops_df[coffee_shops_df['Categories'].apply(lambda x:('Coffee & Tea' or 'Coffee Roasteries' or 'Cafes') in x)]
            
    return coffee_shops_df #.reset_index(drop=True)

In [8]:
def pd_to_gpd(df):
    '''
    Convert coffee df into geodataframe to allow for merge and analysis with neighborhood boundaries from Zillow.
    '''
    #print(df.head())
    geom = df.apply(lambda x: Point(x['Location'][1], x['Location'][0]), axis=1)
    geo_df = gpd.GeoDataFrame(df, geometry=geom)
    geo_df.crs = {'init' :'epsg:4326'}
    return geo_df

In [9]:
coffee_shops_df=find_coffee(loc='Dallas',loops=1)#grab coffee data by city through exhaustive pulls
coffee_shop_gdf=pd_to_gpd(coffee_shops_df)#convert to geo-pandas for easy spatial merging with Zillow
coffee_shops_df.head()

Dallas


,Store_Name,Categories,B_Id,Location,City,Zip_Code,Price,Rating,Review_Count,geometry
0,La La Land Kind Cafe,"[Coffee & Tea, Cafes]",eDft8KCi0nA-tGJFmR7xww,"(32.81528, -96.77114)",Dallas,75206,$,4.5,147,POINT (-96.77114 32.81528)
1,LDU Coffee,[Coffee & Tea],RorY8SkHmDztoyazx_TgPg,"(32.81444, -96.78511)",Dallas,75206,$,5.0,180,POINT (-96.78511 32.81444)
2,Sip Stir Coffee House,[Coffee & Tea],JL4qoK-p4bTdHbuofC3vtA,"(32.8088284, -96.7955301)",Dallas,75204,$$,4.5,452,POINT (-96.79553009999999 32.8088284)
3,Hands+Rose Coffee,[Coffee & Tea],e19NW0QokP6VFAkGVErVRA,"(32.7984520205696, -96.8281446158691)",Dallas,75207,$,4.5,40,POINT (-96.82814461586911 32.7984520205696)
4,Civil Pour,"[Coffee & Tea, Beer Bar, Sandwiches]",YIhFGjkzYVq2QLhpDA1g0Q,"(32.882117, -96.767348)",Dallas,75231,$,4.5,174,POINT (-96.767348 32.882117)


In [10]:
def create_zillow_map(city):
    '''
    Takes the name of a city as an arguement, and pulls the file from Zillow's neighborhood website
    Returns a geopandas dataframe of the city
    '''
    
    state = cities_to_states[city]

    zillow_call = 'https://www.zillowstatic.com/static-neighborhood-boundaries' + \
                  '/LATEST/static-neighborhood-boundaries/shp/' + \
                  'ZillowNeighborhoods-{}.zip'.format(state)

    file_name = '{}_shape.zip'.format(state)
    city_shp = 'ZillowNeighborhoods-{}.shp'.format(state)

    # download the file if it doesn't already exist
    if not os.path.exists(city_shp):
        # download zip from Zillow website
        with urllib.request.urlopen(zillow_call) as response, \
                           open(file_name, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)
        # extract shapefile into current folder
        with zipfile.ZipFile(file_name,"r") as zip_ref:
            zip_ref.extractall()
        # remove unneeded files
        os.remove(file_name)

    data = gpd.read_file(city_shp)
    if city != 'Boston':
        data = data[data['City']==city]
    elif city == 'Boston':
        data = data[(data['City']==city) | (data['City']=='Cambridge') \
                    | (data['City']=='Newton')]
        
    data.crs = {'init' :'epsg:4326'}
    
    return data


In [57]:
# cities to work with for right now more could be added later depending on data available in other domains

cities_to_states = {'Atlanta':'GA',
                    'Austin':'TX',
                    'Baltimore':'MD',
                    'Boston':'MA',
                    'Boulder':'CO',
                    'Charlotte':'NC',
                    'Charlottesville':'VA',
                    'Chicago':'IL',
                    'Cincinnati':'OH',
                    'Cleveland':'OH',
                    'Columbus':'OH',
                    'Dallas':'TX',
                    'Denver':'CO',
                    'Detroit':'MI',
                    'Honolulu':'HI',
                    'Houston':'TX',
                    'Las Vegas':'NV',
                    'Los Angeles':'CA',
                    'Miami':'FL',
                    'Minneapolis':'MN',
                    'New Orleans':'LA',
                    'New York':'NY',
                    'Oakland':'CA',
                    'Philadelphia':'PA',
                    'Pittsburgh':'PA',
                    'Portland':'OR',
                    'Salt Lake City':'UT',
                    'San Diego':'CA',
                    'San Francisco':'CA',
                    'Santa Fe':'NM',
                    'Seattle':'WA',
                    'Washington':'DC',}

cities = sorted(list(cities_to_states.keys()))

# neighborhoods to work with
neighborhoods = pd.read_csv('neighborhoods.csv', index_col=0)
print(np.unique(neighborhoods['City']))

['Atlanta' 'Austin' 'Baltimore' 'Boston' 'Boulder' 'Cambridge' 'Charlotte'
 'Charlottesville' 'Chicago' 'Cleveland' 'Columbus' 'Dallas' 'Denver'
 'Detroit' 'Honolulu' 'Houston' 'Las Vegas' 'Los Angeles' 'Miami'
 'Minneapolis' 'New Orleans' 'New York' 'Newton' 'Oakland' 'Philadelphia'
 'Pittsburgh' 'Portland' 'Salt Lake City' 'San Diego' 'San Francisco'
 'Santa Fe' 'Seattle' 'Washington DC']


In [58]:
sample_df=create_zillow_map('Cincinnati')
sample_df['coords'] = sample_df['geometry'].apply(lambda x: x.representative_point().coords[:])
sample_df['coords'] = [coords[0] for coords in sample_df['coords']]
sample_df.head()

,State,County,City,Name,RegionID,geometry,coords
0,OH,Hamilton,Cincinnati,Corryville,200172,POLYGON ((-84.50155472999987 39.12945585200006...,"(-84.50585524735644, 39.134627310500065)"
3,OH,Hamilton,Cincinnati,Hartwell,201193,POLYGON ((-84.48684399999989 39.22025800000004...,"(-84.46912668871148, 39.211815815149826)"
5,OH,Hamilton,Cincinnati,Bond Hill,199607,POLYGON ((-84.47576123799968 39.19267644300046...,"(-84.46886076671004, 39.18037216550091)"
6,OH,Hamilton,Cincinnati,Camp Washington,199841,POLYGON ((-84.54094143399992 39.15193146000003...,"(-84.54007878797864, 39.13865656750003)"
7,OH,Hamilton,Cincinnati,Clifton,200052,POLYGON ((-84.50746827787883 39.16399381398971...,"(-84.52461637155497, 39.14972841300026)"


In [59]:
def print_cities(cities):#maybe used later
    # print maps of all of the cities
    ncols = 2
    nrows = math.ceil(len(cities)/ncols)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize = (16, 100))
    for i in range(len(cities)):
        city_gdf = create_zillow_map(cities[i])
        ax = plt.subplot(nrows, ncols,i+1)
        city_gdf.plot(figsize=(8,8), color='lightblue', edgecolor='k', ax=ax)
        ax.set_title(cities[i], fontsize=14, weight='bold')
    return
    

In [60]:
def make_giant_gdf(city,loops=5):
    if city == 'Washington':
        city_temp = 'Washington DC'
    else:
        city_temp = city
    coffee_shops_df=find_coffee(loc=city_temp,loops=loops)#grab coffee data by city through exhaustive pulls
    coffee_shop_gdf=pd_to_gpd(coffee_shops_df)#convert to geo-pandas for easy spatial merging with Zillow
    city_map_gdf=create_zillow_map(city)
    #merge the two geopandas dataframe and then do some clean up
    try:
        merged_gdf = gpd.sjoin(coffee_shop_gdf, city_map_gdf, op='within')
    except ValueError:
        print(city_map_gdf.head())
        print('---------')
        print(coffee_shop_gdf.head())
        return city_map_gdf.head(), coffee_shop_gdf.head()
    # do some cleaning and group bys
    merged_gdf.rename(columns={'City_right':'City'}, inplace=True)
    merged_gdf.drop(['index_right', 'City_left'], axis=1, inplace=True)
    merged_gdf.reset_index(drop=True, inplace=True)
    city_map_gdf.reset_index(drop=True, inplace=True)
    #everything below applies to city_map_gdf
    # central coordinates for neighborhood labels
    city_map_gdf['coords'] = city_map_gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
    city_map_gdf['coords'] = [coords[0] for coords in city_map_gdf['coords']]
    # add count of coffee shops in each neighborhood to geo map df
    neighborhood_counts = dict(merged_gdf.groupby('Name').count().iloc[:,0])
    city_map_gdf['count'] = city_map_gdf.Name.map(neighborhood_counts)\
                                        .fillna(0) \
                                        .astype(int)
    # add mean rating to map df
    rating_mean = dict(merged_gdf.groupby('Name').mean()['Rating'])
    city_map_gdf['mean_rating'] = city_map_gdf.Name.map(rating_mean)\
                                              .round(3)
    
    # identify 'chain' coffee stores as those with > 3 location in a city
    # add percentage of stores that are chain for each neighborhood to map df
    chain = dict(merged_gdf['Store_Name'].value_counts() > 3)
    merged_gdf['chain'] = merged_gdf.Store_Name.map(chain)
    pct_chain = dict(merged_gdf.groupby('Name').mean()['chain'])
    city_map_gdf['pct_chain'] = city_map_gdf.Name.map(pct_chain)\
                                                 .astype(float)\
                                                 .round(3)
    # add mean price to map df
    price_map = {'$': 1, '$$': 2, '$$$': 3, '$$$$': 4}
    merged_gdf['price_int'] = merged_gdf.Price.map(price_map)
    price_mean = dict(merged_gdf.groupby('Name').mean()['price_int'])
    city_map_gdf['mean_price'] = city_map_gdf.Name.map(price_mean)\
                                             .round(3)

    # add review count to map df
    total_reviews = dict(merged_gdf.groupby('Name').mean()['Review_Count'])
    city_map_gdf['review_count'] = city_map_gdf.Name.map(total_reviews)\
                                               .fillna(0) \
                                               .astype(int) 
    # identify those shops that only have the category 'Coffee & Tea'
    strictly_coffee = [('Coffee & Tea' in cat and len(cat) == 1) or \
                       ('Cafes' in cat and len(cat) == 1) or \
                       ('Coffee Roasteries' in cat and len(cat) == 1) or \
                       ('Coffee & Tea' in cat and 'Cafes' in cat and len(cat) == 2) or \
                       ('Coffee & Tea' in cat and 'Coffee Roasteries' in cat and len(cat) == 2) or \
                       ('Cafes' in cat and 'Coffee Roasteries' in cat and len(cat) == 2) or \
                       ('Coffee & Tea' in cat and 'Cafes' in cat and 'Coffee Roasteries' in cat and len(cat) == 3) \
                       for cat in merged_gdf.Categories.values]
    merged_gdf['strictly_coffee'] = strictly_coffee
    
    # indentifies whether 'Coffee & Tea' is a category (regardless of if there are others)
    coffee_is_cat = ['Coffee & Tea' in cat or 'Cafes' in cat or 'Coffee Roasteries' in cat \
                     for cat in merged_gdf.Categories.values]
    merged_gdf['coffee_is_cat'] = coffee_is_cat # meow
    
    # add bool columns for other notable categories
    merged_gdf['breakfast_and_brunch'] = ['Breakfast & Brunch' in cat for cat in merged_gdf.Categories.values]
    merged_gdf['bakeries'] = ['Bakeries' in cat for cat in merged_gdf.Categories.values]
    merged_gdf['diner'] = ['Diner' in cat for cat in merged_gdf.Categories.values]
    merged_gdf['deli'] = ['Deli' in cat for cat in merged_gdf.Categories.values]
    merged_gdf['ice_cream_and_froyo'] = ['Ice Cream & Frozen Yogurt' in cat for cat in merged_gdf.Categories.values]
    merged_gdf['juice_and_smoothie'] = ['Juice Bars & Smoothies' in cat for cat in merged_gdf.Categories.values]
    
    # identify primary category of shop, and add a categorical column for it
    def categorize(x):
        '''
        fa la la la la
        '''
        cat_start = x.index.get_loc('chain')
        cat_cols  = x[cat_start:]
        if cat_cols['chain'] == True:
            return 'chain'
        elif cat_cols['diner'] == True:
            return 'diner'
        elif cat_cols['deli'] == True:
            return 'deli'
        elif cat_cols['ice_cream_and_froyo'] == True:
            return 'ice_cream_and_froyo'
        elif cat_cols['juice_and_smoothie'] == True:
            return 'juice_and_smoothie'
        elif cat_cols['bakeries'] == True:
            return 'bakeries'
        elif cat_cols['breakfast_and_brunch'] == True:
            return 'breakfast_and_brunch'
        elif cat_cols['strictly_coffee'] == True:
            return 'strictly_coffee'
        else:
            return 'other'       
    
    merged_gdf['primary_category'] = merged_gdf.apply(lambda x: categorize(x), axis=1)
    
    # now go back and add percentage of our just-defined categories to geo map df,chain already added
    cat_start = merged_gdf.columns.get_loc("strictly_coffee")
    categories = merged_gdf.columns[cat_start:-1]
    for cat in categories:
        try:
            city_map_gdf['pct_' + str(cat)] = city_map_gdf.Name.map(dict( \
                              merged_gdf.groupby('Name').mean()[cat])).round(3)
        except AttributeError:
            city_map_gdf['pct_' + str(cat)] = np.nan
    return city_map_gdf, merged_gdf
    
    
    

In [61]:
def print_top(city_name, df, count=10):
    print('----------------------------------------')
    print('Top {} Categories for {}:'.format(count, city_name))
    print('-----------------------' + '-' * len(city_name))
    df=df[df['City']==city_name]
    print(pd.Series([x for y in df.Categories.values for x in y]).value_counts()[:count])
    print('----------------------------------------')


In [62]:
def plot_map(map_gdf, data_gdf, column='count', neighborhood_labels=True):
    vmin, vmax = map_gdf[column].min(), map_gdf[column].max()
    fig, ax = plt.subplots(1, figsize=(21,15.5))
    base = map_gdf.plot(ax=ax, column='count', edgecolor='white', 
                        cmap='copper', vmin=vmin, vmax=vmax, legend=True)
    data_gdf.plot(ax=base, marker="o", color='r',
                              markersize=10, alpha=0.7)
        # add neighborhood labels
    if neighborhood_labels == True:
        for idx, row in map_gdf.iterrows():
            plt.annotate(s=row['Name'], xy=row['coords'], color='white',
                         horizontalalignment='center', size=8)
    ax.set_title("Coffee Shops in " + str(city), size=20)
    plt.show()

In [63]:
def save_files(list_of_cities, path, num_pulls, update_data=False):
    max_attempts = 5
    if not os.path.isdir(path):
        os.mkdir(path)
    for city in list_of_cities:
        count = 1
        for attempt in range(max_attempts): # Try multiple times as there can be trouble with API calls
            if count>1:
                print('Attempt %i' %count)
            # if the file exists, and we don't want to update data, skip files already downloaded
            # so that we can speed things up.
            if not os.path.isfile(path + str(city) + '/{}_map.csv'.format(city)) or update_data == True:
                try:
                    # run main script
                    city_map_gdf, merged_gdf = make_giant_gdf(city,num_pulls)
                    
                    #print top 10 categories for each, just for the heck of it
                    print_top(city, merged_gdf, 10)
                    print()
                    
                    # save to folder
                    if not os.path.isdir(path + str(city)):
                        os.mkdir(path + str(city))
                    
                    city_map_gdf.geometry.to_file(path + str(city) + '/map_geo_{}.shp'.format(city))
                    map_df = pd.DataFrame(city_map_gdf.values, columns=city_map_gdf.columns)
                    map_df.to_csv(path + str(city) + '/{}_map.csv'.format(city), encoding='utf-8', index=False)
                    merged_gdf.geometry.to_file(path + str(city) + '/data_geo_{}.shp'.format(city))
                    data_df = pd.DataFrame(merged_gdf.values, columns=merged_gdf.columns)
                    data_df.to_csv(path + str(city) + '/{}_data.csv'.format(city), encoding='utf-8', index=False)
                    break
                except:
                    count+=1
                    continue
            if count == (max_attempts+1):
                print('Too many pulls were requested from the API. Lower number and try again.')

In [64]:
# Note that if there is a temporary problem with an API it may give an error below
# Run again and it should be able to make progress

list_of_cities=list(cities_to_states.keys())
num_pulls = 20
max_attempts = 10

print('Below are the cities being scraped for data...')
print(list_of_cities)
print()
path = 'coffee_data/'
# If files already exist delete them before entering definition
if os.path.isdir(path):
    shutil.rmtree(path)

save_files(list_of_cities, path, num_pulls)

Below are the cities being scraped for data...
['Atlanta', 'Austin', 'Baltimore', 'Boston', 'Boulder', 'Charlotte', 'Charlottesville', 'Chicago', 'Cincinnati', 'Cleveland', 'Columbus', 'Dallas', 'Denver', 'Detroit', 'Honolulu', 'Houston', 'Las Vegas', 'Los Angeles', 'Miami', 'Minneapolis', 'New Orleans', 'New York', 'Oakland', 'Philadelphia', 'Pittsburgh', 'Portland', 'Salt Lake City', 'San Diego', 'San Francisco', 'Santa Fe', 'Seattle', 'Washington']

Atlanta
----------------------------------------
Top 10 Categories for Atlanta:
------------------------------
Coffee & Tea                 157
Breakfast & Brunch           120
Cafes                         55
Bakeries                      50
Sandwiches                    43
Desserts                      32
Juice Bars & Smoothies        30
American (Traditional)        25
Ice Cream & Frozen Yogurt     23
Southern                      19
dtype: int64
----------------------------------------

Austin
----------------------------------------

Los Angeles
----------------------------------------
Top 10 Categories for Los Angeles:
----------------------------------
Coffee & Tea                 539
Breakfast & Brunch           200
Bakeries                     123
Cafes                        122
Sandwiches                    90
Desserts                      66
Juice Bars & Smoothies        59
American (New)                40
Donuts                        29
Ice Cream & Frozen Yogurt     26
dtype: int64
----------------------------------------

Miami
----------------------------------------
Top 10 Categories for Miami:
----------------------------
Coffee & Tea                 101
Breakfast & Brunch            80
Cafes                         57
Bakeries                      55
Juice Bars & Smoothies        47
Sandwiches                    44
Desserts                      21
Ice Cream & Frozen Yogurt     20
Cuban                         20
American (New)                15
dtype: int64
----------------------------------------

Mi

In [65]:
def read_in_data(city):
    '''
    reads in files
    '''
    path             = 'coffee_data/'
    crs              = {'init': 'epsg:4326'}
    '''
    WGS84 (EPSG: 4326) +init=epsg:4326 
    +proj=longlat+ellps=WGS84 +
    datum=WGS84 +no_defs+towgs84=0,0,0
    ## CRS used by Google Earth and the U.S. Department of Defense for all their mapping.  
    Tends to be used for global reference systems. GPS satellites broadcast the predicted WGS84 orbits
    '''
    
    def lon_lat_str_to_tuple(string):
        '''
        because coords are converted from tuple to string when 
        '''
        string = string.split(', ')
        lon = float(string[0].split('(')[1])
        lat = float(string[1].split(')')[0])
        return (lon, lat)
    
    # read in map
    map_df           = pd.read_csv(path + str(city) + '/{}_map.csv'.format(city))
    map_df_geo       = gpd.read_file(path + str(city) + '/map_geo_{}.shp'.format(city))
    map_df.geometry  = map_df_geo.geometry
    map_gdf          = gpd.GeoDataFrame(map_df, crs=crs, geometry=map_df.geometry)
    map_gdf.coords   = map_gdf.coords.apply(lambda x: lon_lat_str_to_tuple(x))
    
    # read in data
    data_df          = pd.read_csv(path + str(city) + '/{}_data.csv'.format(city))
    data_df_geo      = gpd.read_file(path + str(city) + '/data_geo_{}.shp'.format(city))
    data_df.geometry = data_df_geo.geometry
    data_gdf         = gpd.GeoDataFrame(data_df, crs=crs, geometry=data_df.geometry)

    return map_gdf, data_gdf

In [66]:
def megalist(list_of_cities):
    
    if len(list_of_cities) == 0:
        print('no cities provided')
        return
    
    # make base dfs
    if not os.path.isfile(path + str(list_of_cities[0]) + '/{}_map.csv'.format(list_of_cities[0])):
        save_files([list_of_cities[0]])
    mega_map, mega_data = read_in_data(list_of_cities[0])
        
    
    if len(list_of_cities) == 1:
        return mega_map, mega_data
    
    for city in list_of_cities[1:]:
        if not os.path.isfile(path + str(city) + '/{}_map.csv'.format(city)):
            save_files([city])
        temp_map, temp_data = read_in_data(city)
        mega_map  = gpd.GeoDataFrame(pd.concat([mega_map, temp_map], ignore_index=True))
        mega_data = gpd.GeoDataFrame(pd.concat([mega_data, temp_data], ignore_index=True))
        
    if not os.path.isdir(path + 'COFFEE'):
        os.mkdir(path + 'COFFEE')
    # save map data
    mega_map.geometry.to_file(path + 'COFFEE' + '/map_geo_{}.shp'.format(city))
    map_df = pd.DataFrame(mega_map.values, columns=mega_map.columns)
    map_df.to_csv(path + 'COFFEE' + '/{}_map.csv'.format('COFFEE'), encoding='utf-8', index=False)
    # save store data
    mega_data.geometry.to_file(path + 'COFFEE' + '/data_geo_{}.shp'.format(city))
    data_df = pd.DataFrame(mega_data.values, columns=mega_data.columns)
    data_df.to_csv(path + 'COFFEE' + '/{}_data.csv'.format('COFFEE'), encoding='utf-8', index=False)
    
    return mega_map, mega_data

In [67]:
import json
from shapely.geometry import Polygon
def read_raw_Uber(city,key_code):
    
    '''
    reads in files
    '''
    path = 'Raw_City_Information/'
    
    city_file_name=path+str(city)+'/{}_taz.json'.format(city.replace(" ", "_"))
    city_file_name
    list_zone_id=[]
    list_poly=[]
    crs = {'init': 'epsg:4326'}
#     print(city_file_name)
    if not os.path.isfile(city_file_name):
        print(str(city.replace(" ", "_"))+'_taz.json does not exist')
    else:
        with open(city_file_name, 'r') as f:
            df=json.load(f)
       #     print(df["features"])
        count=0
        
        for dict_item in df["features"]:
            
            lat_geom_coordinates=[]
            long_geom_coordinates=[]
            dict_zone_id=dict_item["properties"]["MOVEMENT_ID"]
#             print(dict_zone_id)
            list_zone_id.append(dict_zone_id)
            dict_geom_coordinates=dict_item["geometry"]["coordinates"]
#             print(dict_geom_coordinates)
            #create temporary list of lat-long coordinates to convert back to polygon.
            #recall shapely always returns long lat
            if ((city=='Boston') | (city=='Pittsburgh') | (city=='Washington DC')): # These are stored somewhat differently
                long_geom_coordinates=[item[0] for item in dict_geom_coordinates[0][0]]
                lat_geom_coordinates=[item[1] for item in dict_geom_coordinates[0][0]]
            else:
                long_geom_coordinates=[item[0] for item in dict_geom_coordinates[0]]
                lat_geom_coordinates=[item[1] for item in dict_geom_coordinates[0]]
            polygon_geom = Polygon(zip(lat_geom_coordinates, long_geom_coordinates))
            polygon = gpd.GeoDataFrame(crs=crs, geometry=[polygon_geom])
            list_poly.append([polygon.geometry])    
        columns=['dstid','geometry'] 
        data=[list_zone_id,list_poly]
        #make pandas dataframe
        filtered_uber_data=pd.DataFrame(data = dict(zip(columns, data)))
    # avg city mean arrival time to zone id from uber csv.Merge weekday and weekend info
    # as there is no significant difference between the two from previous EDA
    
    if city=='Cincinnati':
        city_uber_file_weekdays=path+str(city)+'/{}-taz2010-2018-{}-OnlyWeekdays-HourlyAggregate.csv'.format(city.lower(),key_code)
    else:
#         print(city)
        city_uber_file_weekdays=path+str(city)+'/{}-taz-2018-{}-OnlyWeekdays-HourlyAggregate.csv'.format(city.replace(" ", "_").lower(),key_code)
    
#     city_uber_file_weekdays=path+str(city)+'/{}-taz-2018-{}-OnlyWeekdays-HourlyAggregate.csv'.format(city.lower(),key_code)
    #city_uber_file_weekends=path+str(city)+'/{}-taz-2018-{}-OnlyWeekends-HourlyAggregate.csv'.format(city.lower(),key_code)
    #merged_uber_pd = pd.read_csv(city_uber_file_weekdays).merge(pd.read_csv(city_uber_file_weekends), on='dstid')
    merged_uber_pd=pd.read_csv(city_uber_file_weekdays)
    filtered_uber_data.dstid=filtered_uber_data.dstid.astype(int)
    pct_chain = dict(merged_uber_pd.groupby('dstid').median()['geometric_mean_travel_time'])
    
    filtered_uber_data['geom_mean_tt_shop'] = filtered_uber_data['dstid'].map(pct_chain)
    temp_list=filtered_uber_data.geometry
    list_comprehension=[item[0][0] for item in temp_list]
    filtered_uber_data['geometry']=list_comprehension
    #print(type(list_comprehension))
    #print(filtered_uber_data.head())
    #try merging with city coffee data if point within poly
    filtered_uber_gdf=gpd.GeoDataFrame(filtered_uber_data, crs=crs, geometry=filtered_uber_data.geometry)
    
    path='coffee_data/'
    if city=='Washington DC':
        city='Washington'
    city_pd=pd.read_csv(path+str(city)+'/{}_data.csv'.format(city))
    city_pd.drop(columns=['geometry'],inplace=True)
    #print(city_pd.head())
    lon=[literal_eval(item)[0] for item in city_pd.Location]
#     print(lon)
    lat=[literal_eval(item)[1] for item in city_pd.Location]
#     print(lat)
    
    
    geom = [Point(xy) for xy in zip(lon, lat)]
    city_gpd = gpd.GeoDataFrame(city_pd, crs=crs, geometry=geom)
#     print(city_gpd.shape)
#     print(filtered_uber_gdf.shape)




    # Check this line
#     print('before merge shape is %s for city and %s for filtered_uber' %(city_gpd.shape,filtered_uber_gdf.shape))
    merged_gdf=gpd.sjoin(city_gpd,filtered_uber_gdf, op='within')
#     print('After merge shape is %s' %(merged_gdf.shape,))
    
    
    
    filtered_uber_gdf.geometry.to_file(path+str(city)+'/data_uber_geo_{}.shp'.format(city))
    data_df = pd.DataFrame(merged_gdf.values, columns=merged_gdf.columns)
    data_df.to_csv(path + str(city) + '/{}_uber_data.csv'.format(city), encoding='utf-8', index=False)
    
    return merged_gdf             
            
     

#     Returns dataframe with only necessary columns, and cleaned in such a way as we will need them later.
#     '''
#     category_list = [[df['categories'][j][i]['title'] \
#     for i in range(len(df['categories'][j]))] \
#     for j in range(len(df['categories']))]

#     location = list(zip(df['coordinates.latitude'], 
#                              df['coordinates.longitude']))

#     columns = ['Store_Name', 'Categories', 'Location', 'City', 
#                    'Zip_Code', 'Price', 'Rating', 'Review_Count']
#     data = [list(df['name']), category_list, location, list(df['location.city']),
#                 list(df['location.zip_code']), list(df['price']), list(df['rating']), 
#                 list(df['review_count'])]


#     filtered_coffee = pd.DataFrame(data = dict(zip(columns, data)))
#     filtered_coffee = filtered_coffee[columns]
#     return filtered_coffee

        
    
    

In [68]:
# Los Angeles is removed as the data was entered incorrectly (this will be adjusted for in the future)

# Also, for now all files from uber with cities with more than one word must be manually renamed
cities_to_states_uber = {'Boston':4 ,'Pittsburgh':3 , 'San Francisco':3 , 'Washington DC':3}
for key,value in cities_to_states_uber.items():
    city=key
    key_code=value
    print('Processing Uber data for ' + city)
    read_raw_Uber(city,int(key_code))

Processing Uber data for Boston
Processing Uber data for Pittsburgh
Processing Uber data for San Francisco
Processing Uber data for Washington DC


In [69]:
#the above cell ends all preliminary scraping and gathering of data.
#I will now compile my megalist